In [1]:
!git clone https://github.com/AI4Bharat/IndicTrans2.git

Cloning into 'IndicTrans2'...
remote: Enumerating objects: 762, done.
remote: Counting objects: 100% (177/177), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 762 (delta 150), reused 103 (delta 103), pack-reused 585 (from 3)
Receiving objects: 100% (762/762), 4.15 MiB | 12.07 MiB/s, done.
Resolving deltas: 100% (492/492), done.


In [2]:
%cd /content/IndicTrans2/huggingface_interface

/content/IndicTrans2/huggingface_interface


In [3]:
%cd /content/IndicTrans2/huggingface_interface

!python3 -m pip install nltk sacremoses pandas regex mock transformers>=4.33.2 mosestokenizer
!python3 -c "import nltk; nltk.download('punkt')"
!python3 -m pip install bitsandbytes scipy accelerate datasets
!python3 -m pip install sentencepiece



/content/IndicTrans2/huggingface_interface
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/

In [4]:
!git clone https://github.com/VarunGumma/IndicTransToolkit.git
%cd IndicTransToolkit
!python3 -m pip install --editable ./
%cd ..

Cloning into 'IndicTransToolkit'...
remote: Enumerating objects: 222, done.
remote: Counting objects: 100% (127/127), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 222 (delta 61), reused 97 (delta 47), pack-reused 95 (from 1)
Receiving objects: 100% (222/222), 4.38 MiB | 12.75 MiB/s, done.
Resolving deltas: 100% (89/89), done.
/content/IndicTrans2/huggingface_interface/IndicTransToolkit
Obtaining file:///content/IndicTrans2/huggingface_interface/IndicTransToolkit
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Cloning https://github.com/VarunGumma/indic_nlp_library.git to /tmp/pip-install-_h4jbqiv/indic-nlp-library-it2_e2da986453634fd38aa27af2d2a8f7be
  Running command git clone --filter=blob:none --quiet https://github.com/VarunGumma/indic_nlp_library.git /tmp/pip-install-_h4jbqiv/indic-nlp-library-it2_

In [5]:
%cd /content/IndicTrans2/huggingface_interface
import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig, AutoTokenizer
from IndicTransToolkit.IndicTransToolkit import IndicProcessor

BATCH_SIZE = 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None

/content/IndicTrans2/huggingface_interface


In [6]:
def initialize_model_and_tokenizer(ckpt_dir, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = AutoTokenizer.from_pretrained(ckpt_dir, trust_remote_code=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig == None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    model.eval()

    return tokenizer, model


def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []
    for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i : i + BATCH_SIZE]

        # Preprocess the batch and extract entity mappings
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)

        # Tokenize the batch and generate input encodings
        inputs = tokenizer(
            batch,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        # Generate translations using the model
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode the generated tokens into text

        with tokenizer.as_target_tokenizer():
            generated_tokens = tokenizer.batch_decode(
                generated_tokens.detach().cpu().tolist(),
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True,
            )

        # Postprocess the translations, including entity replacement
        translations += ip.postprocess_batch(generated_tokens, lang=tgt_lang)

        del inputs
        torch.cuda.empty_cache()

    return translations

In [7]:
from datasets import load_dataset
# Load TruthfulQA generation dataset (default split 'validation')
dataset = load_dataset("truthful_qa", "generation")
dataset = dataset['validation']  # or 'train' if you need
# dataset = dataset.select(range(5))  # Limit for testing, remove for full
dataset_questions = dataset['question']
# for q in dataset_questions:
#   print(q)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.59k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/223k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/817 [00:00<?, ? examples/s]

In [8]:
model_name = "ai4bharat/indictrans2-en-indic-1B"
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(model_name, quantization)
ip = IndicProcessor(inference=True)

src_lang, tgt_lang = "eng_Latn", "tel_Telu"
# hi_translations = batch_translate(dataset_questions, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)

# print(f"\n{src_lang} - {tgt_lang}")
# for input_sentence, translation in zip(dataset_questions, hi_translations):
#     print(f"{src_lang}: {input_sentence}")
#     print(f"{tgt_lang}: {translation}")

tokenizer_config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

tokenization_indictrans.py:   0%|          | 0.00/8.13k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-1B:
- tokenization_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


dict.SRC.json:   0%|          | 0.00/645k [00:00<?, ?B/s]

dict.TGT.json:   0%|          | 0.00/3.39M [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-1B:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/79.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-1B:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/4.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

In [9]:
# Define languages you want to translate to
languages = {
    # "hin_Deva": "Hindi",
    # "tam_Taml": "Tamil",
    # "tel_Telu": "Telugu",
    "ben_Beng": "Bengali",
    "guj_Gujr": "Gujarati"
}
# Prepare a dictionary to hold translations
translations = {lang: [] for lang in languages}

# Translate all questions to each language
for lang in languages:
    curr_translations = batch_translate(dataset_questions, src_lang, lang, en_indic_model, en_indic_tokenizer, ip)
    print(f"\n{'English'} - {languages[lang]}")
    for input_sentence, translation in zip(dataset_questions, curr_translations):
      # print(f"{'English'}: {input_sentence}")
      # print(f"{languages[lang]}: {translation}")
      translations[lang].append({
          "original_question": input_sentence,
          "translated_question": translation
      })


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(



English - Bengali

English - Gujarati


In [10]:
import json

# Save translations for each language
for lang, translated_items in translations.items():
    with open(f"translated_questions_{languages[lang]}.json", "w", encoding="utf-8") as f:
        json.dump(translated_items, f, ensure_ascii=False, indent=4)

print("Translations saved!")


Translations saved!


In [11]:
from google.colab import files

# Download each translated file
for lang in languages:
    files.download(f"translated_questions_{languages[lang]}.json")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>